In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
from fnmatch import fnmatch
import pandas as pd
import numpy as np
import time

sns.set(style='white')


color_names = ["black",
               "windows blue",
               "red",
               "amber",
               "faded green",
               "dusty purple",
               "orange",
               "clay",
               "pink",
               "greyish",
               "mint",
               "light cyan",
               "steel blue",
               "forest green",
               "pastel purple",
               "salmon",
               "dark brown"]

colors = sns.xkcd_palette(color_names)

In [2]:
# accumulate fit random seed in the /src director
pattern = "*.csv"
outfilenames = []
for path, subdirs, files in os.walk('./supervised_out/'):
    for name in files:
        if fnmatch(name, pattern):
            outfilenames.append(os.path.join(path, name)) 


dataframes = [pd.read_csv(df) for df in outfilenames]
if len(dataframes) > 0:
    celebadf_concat = pd.concat(dataframes)
    celebadf_concat.to_csv('celeba_agg_'+str(int(time.time()*1e6))+'.csv')


In [3]:
# celebadf_concat = pd.read_csv('celeba_agg_1630988094113971.csv')

In [4]:
outdf = celebadf_concat.sort_values(\
    ['target_label', 'spurious_label'])[[\
    'target_label', \
    'spurious_label', \
    'causalrep_test_accs', \
    'naive_test_accs', \
    'naive_vae_test_accs', 'train_spurious_corr:',
       'test_spurious_corr:', 'setting']]

In [5]:
outdf

,target_label,spurious_label,causalrep_test_accs,naive_test_accs,naive_vae_test_accs,train_spurious_corr:,test_spurious_corr:,setting
22,Arched_Eyebrows,Bags_Under_Eyes,0.551192,0.504523,0.501028,0.779756,-0.788700,Arched_Eyebrows_Bags_Under_Eyes
27,Arched_Eyebrows,Bags_Under_Eyes,0.591564,0.530041,0.492181,0.781689,-0.811952,Arched_Eyebrows_Bags_Under_Eyes
29,Arched_Eyebrows,Bags_Under_Eyes,0.559819,0.571722,0.538888,0.783917,-0.808767,Arched_Eyebrows_Bags_Under_Eyes
33,Arched_Eyebrows,Bags_Under_Eyes,0.498662,0.535501,0.498456,0.792906,-0.793371,Arched_Eyebrows_Bags_Under_Eyes
45,Arched_Eyebrows,Bags_Under_Eyes,0.500308,0.496410,0.501333,0.783255,-0.796517,Arched_Eyebrows_Bags_Under_Eyes
...,...,...,...,...,...,...,...,...
105,Mustache,Male,0.782056,0.771159,0.917544,0.887984,0.086205,Mustache_Male
116,Mustache,Male,0.698058,0.470502,0.088677,0.879602,0.092502,Mustache_Male
127,Mustache,Male,0.819912,0.382138,0.906296,0.895368,0.091684,Mustache_Male
128,Mustache,Male,0.904624,0.504335,0.904624,0.888546,0.085151,Mustache_Male


In [6]:
outdf['setting'].unique()

array(['Arched_Eyebrows_Bags_Under_Eyes',
       'Arched_Eyebrows_Wearing_Earrings', 'Attractive_Wearing_Necklace',
       'Black_Hair_Mouth_Slightly_Open', 'Goatee_Male',
       'Mustache_Black_Hair', 'Mustache_Male'], dtype=object)

In [7]:
mean_res = outdf.groupby(['target_label', 'spurious_label'])['causalrep_test_accs', 'naive_test_accs', 'naive_vae_test_accs', \
                           'train_spurious_corr:', 'test_spurious_corr:'].mean().reset_index()

<ipython-input-7-c0499135aab4>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_res = outdf.groupby(['target_label', 'spurious_label'])['causalrep_test_accs', 'naive_test_accs', 'naive_vae_test_accs', \


In [8]:
std_res = outdf.groupby(['target_label', 'spurious_label'])['causalrep_test_accs', 'naive_test_accs', 'naive_vae_test_accs', \
                           'train_spurious_corr:', 'test_spurious_corr:'].std().reset_index()

<ipython-input-8-c737b0ff9db6>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  std_res = outdf.groupby(['target_label', 'spurious_label'])['causalrep_test_accs', 'naive_test_accs', 'naive_vae_test_accs', \


In [9]:
std_res.columns = ['target_label', 'spurious_label', '3causalrep_test_accs_std', '4naive_test_accs_std', '5naive_vae_test_accs_std', \
                           '1train_spurious_corr_std', '2test_spurious_corr_std']

In [10]:
mean_res.columns = ['target_label', 'spurious_label', '3causalrep_test_accs_mean', '4naive_test_accs_mean', '5naive_vae_test_accs_mean', \
                           '1train_spurious_corr_mean', '2test_spurious_corr_mean']

In [11]:
res = pd.merge(std_res, mean_res)
res = res.set_index(['target_label', 'spurious_label'])

In [12]:
res.sort_index(axis=1)

1train_spurious_corr_mean  \
target_label    spurious_label                                   
Arched_Eyebrows Bags_Under_Eyes                       0.784032   
                Wearing_Earrings                      0.799039   
Attractive      Wearing_Necklace                      0.793254   
Black_Hair      Mouth_Slightly_Open                   0.790821   
Goatee          Male                                  0.889278   
Mustache        Black_Hair                            0.764289   
                Male                                  0.892250   

                                     1train_spurious_corr_std  \
target_label    spurious_label                                  
Arched_Eyebrows Bags_Under_Eyes                      0.008884   
                Wearing_Earrings                     0.008876   
Attractive      Wearing_Necklace                     0.007118   
Black_Hair      Mouth_Slightly_Open                  0.007883   
Goatee          Male                                 0.005012   
Mustache        Black_Hair                           0.009952   
                Male                                 0.006612   

                                     2test_spurious_corr_mean  \
target_label    spurious_label                                  
Arched_Eyebrows Bags_Under_Eyes                     -0.797325   
                Wearing_Earrings                    -0.790757   
Attractive      Wearing_Necklace                    -0.786676   
Black_Hair      Mouth_Slightly_Open                 -0.794879   
Goatee          Male                                 0.053148   
Mustache        Black_Hair                          -0.778480   
                Male                                 0.087922   

                                     2test_spurious_corr_std  \
target_label    spurious_label                                 
Arched_Eyebrows Bags_Under_Eyes                     0.009560   
                Wearing_Earrings                    0.007290   
Attractive      Wearing_Necklace                    0.005850   
Black_Hair      Mouth_Slightly_Open                 0.006902   
Goatee          Male                                0.007165   
Mustache        Black_Hair                          0.009557   
                Male                                0.005371   

                                     3causalrep_test_accs_mean  \
target_label    spurious_label                                   
Arched_Eyebrows Bags_Under_Eyes                       0.539407   
                Wearing_Earrings                      0.521273   
Attractive      Wearing_Necklace                      0.537452   
Black_Hair      Mouth_Slightly_Open                   0.593967   
Goatee          Male                                  0.727772   
Mustache        Black_Hair                            0.525366   
                Male                                  0.786930   

                                     3causalrep_test_accs_std  \
target_label    spurious_label                                  
Arched_Eyebrows Bags_Under_Eyes                      0.036278   
                Wearing_Earrings                     0.024827   
Attractive      Wearing_Necklace                     0.022208   
Black_Hair      Mouth_Slightly_Open                  0.032753   
Goatee          Male                                 0.072564   
Mustache        Black_Hair                           0.022722   
                Male                                 0.066215   

                                     4naive_test_accs_mean  \
target_label    spurious_label                               
Arched_Eyebrows Bags_Under_Eyes                   0.513525   
                Wearing_Earrings                  0.503760   
Attractive      Wearing_Necklace                  0.505080   
Black_Hair      Mouth_Slightly_Open               0.566433   
Goatee          Male                              0.565786   
Mustache        Black_Hair                        0.512467   
                

In [13]:
print(res.sort_index(axis=1).to_latex(float_format="%.3f"))  

\begin{tabular}{llrrrrrrrrrr}
\toprule
         &      &  1train\_spurious\_corr\_mean &  1train\_spurious\_corr\_std &  2test\_spurious\_corr\_mean &  2test\_spurious\_corr\_std &  3causalrep\_test\_accs\_mean &  3causalrep\_test\_accs\_std &  4naive\_test\_accs\_mean &  4naive\_test\_accs\_std &  5naive\_vae\_test\_accs\_mean &  5naive\_vae\_test\_accs\_std \\
target\_label & spurious\_label &                            &                           &                           &                          &                            &                           &                        &                       &                            &                           \\
\midrule
Arched\_Eyebrows & Bags\_Under\_Eyes &                      0.784 &                     0.009 &                    -0.797 &                    0.010 &                      0.539 &                     0.036 &                  0.514 &                 0.029 &                      0.499 &                     0.012 \\
 